In [1]:
import urllib.request, json , time, os, difflib, itertools
import pandas as pd
from multiprocessing.dummy import Pool
from datetime import datetime


In [2]:
try:
    import httplib
except:
    import http.client as httplib

def check_internet():
    conn = httplib.HTTPConnection("www.google.com", timeout=5)
    try:
        conn.request("HEAD", "/")
        conn.close()
        # print("True")
        return True
    except:
        conn.close()
        # print("False")
        return False

In [3]:
check_internet()

True

In [4]:
def get_historic_price(query_url,json_path,csv_path):
    
    while not check_internet():
        print("Could not connect, trying again in 5 seconds...")
        time.sleep(5)
    
    stock_id=query_url.split("&period")[0].split("symbol=")[1]
    
    if os.path.exists(csv_path+stock_id+'.csv') and os.stat(csv_path+stock_id+'.csv').st_size != 0:
        print("<<<  Historical data of "+stock_id+" already exists, Updating data...")

    try:
        with urllib.request.urlopen(query_url) as url:
            parsed = json.loads(url.read().decode())
    except:
        print("|||  Historical data of "+stock_id+" doesn't exist")
        return
    
    else:
        if os.path.exists(json_path+stock_id+'.json'):
            os.remove(json_path+stock_id+'.json')
        with open(json_path+stock_id+'.json', 'w') as outfile:
            json.dump(parsed, outfile, indent=4)

        try:
            Date=[]
            for i in parsed['chart']['result'][0]['timestamp']:
                Date.append(datetime.utcfromtimestamp(int(i)).strftime('%d-%m-%Y'))

            Low=parsed['chart']['result'][0]['indicators']['quote'][0]['low']
            Open=parsed['chart']['result'][0]['indicators']['quote'][0]['open']
            Volume=parsed['chart']['result'][0]['indicators']['quote'][0]['volume']
            High=parsed['chart']['result'][0]['indicators']['quote'][0]['high']
            Close=parsed['chart']['result'][0]['indicators']['quote'][0]['close']
            Adjusted_Close=parsed['chart']['result'][0]['indicators']['adjclose'][0]['adjclose']

            df=pd.DataFrame(list(zip(Date,Low,Open,Volume,High,Close,Adjusted_Close)),columns =['Date','Low','Open','Volume','High','Close','Adjusted Close'])

            if os.path.exists(csv_path+stock_id+'.csv'):
                os.remove(csv_path+stock_id+'.csv')
            df.to_csv(csv_path+stock_id+'.csv', sep=',', index=None)
            print(">>>  Historical data of "+stock_id+" saved")
            return
        except:
            print(">>>  Historical data of "+stock_id+" exists but has no trading data")

In [5]:
json_path = os.getcwd()+os.sep+".."+os.sep+"historic_data"+os.sep+"json"+os.sep
csv_path = os.getcwd()+os.sep+".."+os.sep+"historic_data"+os.sep+"csv"+os.sep


In [6]:
if not os.path.isdir(json_path):
    os.makedirs(json_path)
if not os.path.isdir(csv_path):
    os.makedirs(csv_path)


In [7]:
period1 = 0
period2 = 9999999999
interval = "1d"

In [8]:
country_name = "germany"


In [11]:
ticker_file_path = "../data"+os.sep+"Yahoo Ticker Symbols - September 2017.xlsx"
ticker_file_path

'../data/Yahoo Ticker Symbols - September 2017.xlsx'

In [12]:
temp_df = pd.read_excel(ticker_file_path)
print("Total stocks:",len(temp_df))
temp_df.head(10)

Total stocks: 106331


,Yahoo Stock Tickers,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,http://investexcel.net,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Ticker,Name,Exchange,Category Name,Country,NaN,NaN,NaN
3,OEDV,"Osage Exploration and Development, Inc.",PNK,NaN,USA,NaN,NaN,Samir Khan
4,AAPL,Apple Inc.,NMS,Electronic Equipment,USA,NaN,NaN,simulationconsultant@gmail.com
5,BAC,Bank of America Corporation,NYQ,Money Center Banks,USA,NaN,NaN,NaN
6,AMZN,"Amazon.com, Inc.",NMS,Catalog & Mail Order Houses,USA,NaN,NaN,This ticker symbol list was downloaded from
7,T,AT&T Inc.,NYQ,Telecom Services - Domestic,USA,NaN,NaN,http://investexcel.net/all-yahoo-finance-stock...
8,GOOG,Alphabet Inc.,NMS,Internet Information Providers,USA,NaN,NaN,and was updated on 2nd September 2017
9,MO,"Altria Group, Inc.",NYQ,Cigarettes,USA,NaN,NaN,NaN


In [13]:
temp_df = temp_df.drop(temp_df.columns[[5, 6, 7]], axis=1)
headers = temp_df.iloc[2]
df  = pd.DataFrame(temp_df.values[3:], columns=headers)
print("Total stocks:",len(df))
df.head(10)

Total stocks: 106328


2,Ticker,Name,Exchange,Category Name,Country
0,OEDV,"Osage Exploration and Development, Inc.",PNK,NaN,USA
1,AAPL,Apple Inc.,NMS,Electronic Equipment,USA
2,BAC,Bank of America Corporation,NYQ,Money Center Banks,USA
3,AMZN,"Amazon.com, Inc.",NMS,Catalog & Mail Order Houses,USA
4,T,AT&T Inc.,NYQ,Telecom Services - Domestic,USA
5,GOOG,Alphabet Inc.,NMS,Internet Information Providers,USA
6,MO,"Altria Group, Inc.",NYQ,Cigarettes,USA
7,DAL,"Delta Air Lines, Inc.",NYQ,Major Airlines,USA
8,AA,Alcoa Corporation,NYQ,Aluminum,USA
9,AXP,American Express Company,NYQ,Credit Services,USA


In [14]:
new_df = df[df["Country"].str.lower().str.contains(country_name.lower()) == True]
print("Total stocks:",len(new_df))
new_df.head(10)

Total stocks: 21365


2,Ticker,Name,Exchange,Category Name,Country
400,ADS.DE,Adidas AG,GER,Textile - Apparel Footwear & Accessories,Germany
639,BAYN.DE,Bayer Aktiengesellschaft,GER,Drug Manufacturers - Major,Germany
723,DAI.DE,Daimler AG,GER,Auto Manufacturers - Major,Germany
974,WDI.DE,Wirecard AG,GER,Business Services,Germany
1021,ALV.DE,Allianz SE,GER,Property & Casualty Insurance,Germany
1175,LO3.DE,Lonza Group Ltd,GER,Medical Laboratories & Research,Germany
1186,IVX.DE,InVision Aktiengesellschaft,GER,Business Software & Services,Germany
1204,RSTA.DE,RIB Software AG,GER,NaN,Germany
1308,VWS.DE,Vestas Wind Systems A/S,GER,Diversified Machinery,Germany
1331,RKET.DE,Rocket Internet SE,GER,Technical & System Software,Germany


In [15]:
new_df.to_csv('../data'+os.sep+country_name+'.csv', sep=',', index=None)


In [16]:
dax_40_companies = [
    "Adidas",
    "Allianz",
    "BASF",
    "Bayer",
    "BMW",
    "Brenntag",
    "Continental",
    "Covestro",
    "Daimler",
    "Delivery Hero",
    "Deutsche Bank",
    "Deutsche Börse",
    "Deutsche Post",
    "Deutsche Telekom",
    "E.ON",
    "Fresenius Medical Care",
    "Fresenius",
    "Hannover Rück",
    "HeidelbergCement",
    "HelloFresh",
    "Henkel",
    "Infineon Technologies",
    "Linde",
    "Mercedes",
    "Merck",
    "MTU Aero Engines",
    "Münchener Rückversicherungs-Gesellschaft",
    "Porsche",
    "Puma",
    "RWE",
    "Qiagen",
    "SAP",
    "Sartorius",
    "Siemens",
    "Siemens Healthineers",
    "Symrise",
    "ThyssenKrupp",
    "Volkswagen",
    "Vonovia",
    "Zalando",
]

In [17]:
len(dax_40_companies)

40

In [18]:
ticker_list=[]
for company in dax_40_companies:
    try:
        exact_company_name = (difflib.get_close_matches(company, new_df['Name'])[0])
        ticker_for_the_company = new_df.loc[new_df['Name'] == exact_company_name, 'Ticker'].iloc[0]
        ticker_list.append(ticker_for_the_company)
    except:
        print("Company name "+company+" not found.")
ticker_list

Company name Bayer not found.
Company name BMW not found.
Company name Continental not found.
Company name Covestro not found.
Company name Delivery Hero not found.
Company name Fresenius not found.
Company name HelloFresh not found.
Company name Henkel not found.
Company name Linde not found.
Company name Mercedes not found.
Company name Merck not found.
Company name Porsche not found.
Company name Puma not found.
Company name RWE not found.
Company name Qiagen not found.
Company name Siemens not found.
Company name Siemens Healthineers not found.
Company name Volkswagen not found.


['ADS.DE',
 'ALV.DE',
 'BAS.DE',
 'BNR.DE',
 'DAI.DE',
 'DPB.DE',
 'DPW.DE',
 'DPW.DE',
 'DTE.DE',
 'EOAN.DE',
 'FMEA.SG',
 'HNRB.DE',
 'HEI.DE',
 'IFX.DE',
 'MTX.DE',
 'MUV2.DE',
 'SRP.BE',
 'BGZ.HM',
 'SY1.DE',
 'TKA.DE',
 'VNA.DE',
 'ZAL.DE']

In [19]:
manually_added=["BAYN.DE","BMW.DE","CON.DE","1COV.DE","DHER.DE","FRE.DE","HFG.DE","HEN3.DE","LIN.DE",
 "MBG.DE","MRK.DE","P911.DE","PUM.DE","RWE.DE","QIA.DE","SIE.DE","SHL.DE","VOW3.DE"]

In [20]:
full_ticker_list=ticker_list+manually_added
full_ticker_list

['ADS.DE',
 'ALV.DE',
 'BAS.DE',
 'BNR.DE',
 'DAI.DE',
 'DPB.DE',
 'DPW.DE',
 'DPW.DE',
 'DTE.DE',
 'EOAN.DE',
 'FMEA.SG',
 'HNRB.DE',
 'HEI.DE',
 'IFX.DE',
 'MTX.DE',
 'MUV2.DE',
 'SRP.BE',
 'BGZ.HM',
 'SY1.DE',
 'TKA.DE',
 'VNA.DE',
 'ZAL.DE',
 'BAYN.DE',
 'BMW.DE',
 'CON.DE',
 '1COV.DE',
 'DHER.DE',
 'FRE.DE',
 'HFG.DE',
 'HEN3.DE',
 'LIN.DE',
 'MBG.DE',
 'MRK.DE',
 'P911.DE',
 'PUM.DE',
 'RWE.DE',
 'QIA.DE',
 'SIE.DE',
 'SHL.DE',
 'VOW3.DE']

In [21]:
full_ticker_dict={
"Adidas":'ADS.DE',
 "Airbus":'AIR.PA',
 "Allianz":'ALV.DE',
 "BASF":'BAS.DE',
"Bayer":'BAYN.DE',
 "BMW":'BMW.DE',
 "Brenntag":'BNR.DE',
"Continental":'CON.DE',
    "Covestro": '1COV.DE',
    "Daimler Truck":'DTG.DE',
    "Delivery Hero": 'DHER.DE',
     "Deutsche Börse":'DB1.DE',
    "Deutsche Bank":"DBK.DE",
 "Deutsche Post": 'DHL.DE',
 "Deutsche Telekom":'DTE.DE',
 "E.ON":'EOAN.DE',
     "Fresenius": "FRE.DE",
    "Fresenius Medical Care":'FME.DE',

 "Hannover Rück":'HNR1.DE',
 "HeidelbergCement": 'HEI.DE',
    "Hello Fresh": 'HFG.DE',
     "Henkel":'HEN3.DE',
    "Infineon": 'IFX.DE',
"Linde":'LIN.DE',
"Mercedes Benz": "MBG.DE",


    "Merck": 'MRK.DE',
 "MTU Aero Engines":'MTX.DE',
 "Münchner Rück":'MUV2.DE',
     "Porsche":'P911.DE',
     "Puma":'PUM.DE',
     "QIAGEN":'QIA.DE',
    "RWE": 'RWE.DE',
     "SAP":'SAP.DE',
 "Sartorius":'SRT3.DE',
    "Siemens": 'SIE.DE',
    "Siemens Healthineers": 'SHL.DE',
"Symrise":'SY1.DE',
    "Volkswagen":'VOW3.DE',
"Vonovia": 'VNA.DE',
 "Zalando":'ZAL.DE',}

In [22]:
len(full_ticker_dict)

40

In [23]:
full_ticker_dict.values()

dict_values(['ADS.DE', 'AIR.PA', 'ALV.DE', 'BAS.DE', 'BAYN.DE', 'BMW.DE', 'BNR.DE', 'CON.DE', '1COV.DE', 'DTG.DE', 'DHER.DE', 'DB1.DE', 'DBK.DE', 'DHL.DE', 'DTE.DE', 'EOAN.DE', 'FME.DE', 'FMEA.SG', 'HNRB.DE', 'HEI.DE', 'HFG.DE', 'HEN3.DE', 'IFX.DE', 'LIN.DE', 'MBG.DE', 'MRK.DE', 'MTX.DE', 'MUV2.DE', 'P911.DE', 'PUM.DE', 'QIA.DE', 'RWE.DE', 'SAP.DE', 'SRP.DE', 'SIE.DE', 'SHL.DE', 'SY1.DE', 'VOW3.DE', 'VNA.DE', 'ZAL.DE'])

In [24]:
query_urls=[]
for ticker in full_ticker_dict.values():
    query_urls.append("https://query1.finance.yahoo.com/v8/finance/chart/"+ticker+"?symbol="+ticker+"&period1=0&period2=9999999999&interval=1d&includePrePost=true&events=div%2Csplit")

In [25]:
with Pool(processes=len(query_urls)) as pool:
    pool.starmap(get_historic_price, zip(query_urls, itertools.repeat(json_path), itertools.repeat(csv_path)))
print("All downloads completed !")

>>>  Historical data of P911.DE saved
>>>  Historical data of HNRB.DE saved
>>>  Historical data of DTG.DE saved
>>>  Historical data of SRP.DE saved
>>>  Historical data of HFG.DE saved
>>>  Historical data of SHL.DE saved
>>>  Historical data of DHER.DE saved
>>>  Historical data of ZAL.DE saved
>>>  Historical data of 1COV.DE saved
>>>  Historical data of BNR.DE saved
>>>  Historical data of VNA.DE saved
>>>  Historical data of MTX.DE saved
>>>  Historical data of FMEA.SG saved
>>>  Historical data of SY1.DE saved
>>>  Historical data of LIN.DE saved
>>>  Historical data of MUV2.DE saved
>>>  Historical data of DHL.DE saved>>>  Historical data of DB1.DE saved

>>>  Historical data of HEN3.DE saved
>>>  Historical data of EOAN.DE saved
>>>  Historical data of RWE.DE saved
>>>  Historical data of MRK.DE saved
>>>  Historical data of ALV.DE saved
>>>  Historical data of IFX.DE saved
>>>  Historical data of DTE.DE saved
>>>  Historical data of BMW.DE saved>>>  Historical data of AIR.PA 

In [26]:
df_translate=pd.DataFrame(full_ticker_dict,index=[0]).T.reset_index()
df_translate.columns=["name","shortener"]
df_translate

,name,shortener
0,Adidas,ADS.DE
1,Airbus,AIR.PA
2,Allianz,ALV.DE
3,BASF,BAS.DE
4,Bayer,BAYN.DE
5,BMW,BMW.DE
6,Brenntag,BNR.DE
7,Continental,CON.DE
8,Covestro,1COV.DE
9,Daimler Truck,DTG.DE


In [78]:
df_translate.to_csv("data/translate.csv",index=False)